In [14]:
# Import Necessary Libraries
# !pip install openai==0.28.1

import numpy as np
import pandas as pd
import random
import openai
import time # for error handling
import re


API_KEY = ''


df = pd.read_csv('../../data/sampled_tweets_2000_copy.csv')

In [4]:
display(df)

,created_at_fuzzy,state_likely_tweeted_from,text,source,display_text_width,lang,favorite_count,retweet_count,is_quote,followers_count,friends_count,listed_count,statuses_count,favourites_count
0,2020-08-13T16:55:49Z,California,LIVE: Trump holds press conference https://t.c...,Twitter Web App,71,en,0,0,False,3281,4938,4,39349,23299
1,2020-08-30T00:03:31Z,Tennessee,Trump’s claim the ‘silent majority’ will save ...,Twitter Web App,205,en,9,2,False,2866,3000,2,23786,30906
2,2020-10-26T23:16:05Z,New York,#NewHampshire #VoteBlue #VoteBlueToSaveAmerica...,Twitter Web App,267,en,7,4,False,32221,26550,52,304911,146137
3,2020-08-24T09:54:59Z,Florida,"@davidsirota I'm so confused, when is the medi...",Twitter for Android,262,en,2,0,False,457,375,5,16851,7429
4,2021-02-04T11:31:07Z,Washington,@Carlos27480217 @DonaldJTrumpJr Yep! He has Ba...,Twitter for Android,88,en,0,0,False,43,177,0,2618,1517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2021-01-15T18:46:33Z,Georgia,U.S. says Capitol rioters meant to 'capture an...,Twitter for iPhone,126,en,2,0,False,1546,1600,2,5156,3058
1996,2020-12-21T13:11:53Z,Iowa,@KeithOlbermann Look in Keith’s eyes. Trump o...,Twitter for iPhone,104,en,0,0,False,28,50,0,758,6974
1997,2021-02-06T06:35:34Z,North Carolina,@AmyAThatcher I guess the right wingers are pi...,Twitter for Android,125,en,1,0,False,598,512,9,26537,156580
1998,2020-08-14T07:37:59Z,Iowa,Insane....\n\nBiden calls for mandate requirin...,Twitter for iPad,98,en,1,0,False,31,96,0,1041,2414


In [15]:
for index, row in df.head(60).iterrows():
    print(row['text'])

LIVE: Trump holds press conference https://t.co/6vWNh32OxY via @YouTube
Trump’s claim the ‘silent majority’ will save his re-election bid demolished by CNN polling expert...

Because no matter how much Trump wishes it were true, the voices in his head won't get him re-elected.
#NewHampshire #VoteBlue #VoteBlueToSaveAmerica  #VoteBlueToSaveAmerica2020 

#BIDENHARRIS2020

This influential newspaper just broke a 100-plus year streak of endorsing a Republican and endorsed JOE BIDEN FOR PRESIDENT!
https://t.co/mS86DxsHHQ

#NHpol #NHpolitics #NH
@davidsirota I'm so confused, when is the media going to point out the obvious and brutal attacks by Kamala Harris supporters and the Joe Biden supporters? Why was Bernie Sanders supporters smeared just because they critiqued using policy? I hate the media hypocrisy on this.
@Carlos27480217 @DonaldJTrumpJr Yep! He has Balls!! Now with Our Whimp in office and Biden will have US owned by China!!
@HouseGOP @SenateGOP @GOP @NRSC @NRCC It is in the best i

In [16]:
biden_variations = {
    r'\bJOE BIDEN\b': 'presidential candidate',
    r'\bJoe Biden\b': 'presidential candidate',
    r'\bBiden\b': 'presidential candidate',
    r'\bBIDEN\b': 'presidential candidate',
}

In [17]:


# compiling regex patterns
patterns = {re.compile(k): v for k, v in biden_variations.items()}

def replace_and_binary(text):
    biden_mentioned = 0
    for pattern, replacement in patterns.items():
        if pattern.search(text):
            biden_mentioned = 1
            text = pattern.sub(replacement, text)
    return biden_mentioned, text

# applying the function to the "text" column
df['Biden initially'], df['text'] = zip(*df['text'].apply(replace_and_binary))



          created_at_fuzzy state_likely_tweeted_from  \
0     2020-08-13T16:55:49Z                California   
1     2020-08-30T00:03:31Z                 Tennessee   
2     2020-10-26T23:16:05Z                  New York   
3     2020-08-24T09:54:59Z                   Florida   
4     2021-02-04T11:31:07Z                Washington   
...                    ...                       ...   
1995  2021-01-15T18:46:33Z                   Georgia   
1996  2020-12-21T13:11:53Z                      Iowa   
1997  2021-02-06T06:35:34Z            North Carolina   
1998  2020-08-14T07:37:59Z                      Iowa   
1999  2020-09-13T10:35:06Z                   Florida   

                                                   text               source  \
0     LIVE: Trump holds press conference https://t.c...      Twitter Web App   
1     Trump’s claim the ‘silent majority’ will save ...      Twitter Web App   
2     #NewHampshire #VoteBlue #VoteBlueToSaveAmerica...      Twitter Web App   
3     @

In [18]:
display(df)

,created_at_fuzzy,state_likely_tweeted_from,text,source,display_text_width,lang,favorite_count,retweet_count,is_quote,followers_count,friends_count,listed_count,statuses_count,favourites_count,Biden initially
0,2020-08-13T16:55:49Z,California,LIVE: Trump holds press conference https://t.c...,Twitter Web App,71,en,0,0,False,3281,4938,4,39349,23299,0
1,2020-08-30T00:03:31Z,Tennessee,Trump’s claim the ‘silent majority’ will save ...,Twitter Web App,205,en,9,2,False,2866,3000,2,23786,30906,0
2,2020-10-26T23:16:05Z,New York,#NewHampshire #VoteBlue #VoteBlueToSaveAmerica...,Twitter Web App,267,en,7,4,False,32221,26550,52,304911,146137,1
3,2020-08-24T09:54:59Z,Florida,"@davidsirota I'm so confused, when is the medi...",Twitter for Android,262,en,2,0,False,457,375,5,16851,7429,1
4,2021-02-04T11:31:07Z,Washington,@Carlos27480217 @DonaldJTrumpJr Yep! He has Ba...,Twitter for Android,88,en,0,0,False,43,177,0,2618,1517,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2021-01-15T18:46:33Z,Georgia,U.S. says Capitol rioters meant to 'capture an...,Twitter for iPhone,126,en,2,0,False,1546,1600,2,5156,3058,0
1996,2020-12-21T13:11:53Z,Iowa,@KeithOlbermann Look in Keith’s eyes. Trump o...,Twitter for iPhone,104,en,0,0,False,28,50,0,758,6974,0
1997,2021-02-06T06:35:34Z,North Carolina,@AmyAThatcher I guess the right wingers are pi...,Twitter for Android,125,en,1,0,False,598,512,9,26537,156580,1
1998,2020-08-14T07:37:59Z,Iowa,Insane....\n\npresidential candidate calls for...,Twitter for iPad,98,en,1,0,False,31,96,0,1041,2414,1


In [19]:
for index, row in df.head(10).iterrows():
    print(row['text'])

LIVE: Trump holds press conference https://t.co/6vWNh32OxY via @YouTube
Trump’s claim the ‘silent majority’ will save his re-election bid demolished by CNN polling expert...

Because no matter how much Trump wishes it were true, the voices in his head won't get him re-elected.
#NewHampshire #VoteBlue #VoteBlueToSaveAmerica  #VoteBlueToSaveAmerica2020 

#BIDENHARRIS2020

This influential newspaper just broke a 100-plus year streak of endorsing a Republican and endorsed presidential candidate FOR PRESIDENT!
https://t.co/mS86DxsHHQ

#NHpol #NHpolitics #NH
@davidsirota I'm so confused, when is the media going to point out the obvious and brutal attacks by Kamala Harris supporters and the presidential candidate supporters? Why was Bernie Sanders supporters smeared just because they critiqued using policy? I hate the media hypocrisy on this.
@Carlos27480217 @DonaldJTrumpJr Yep! He has Balls!! Now with Our Whimp in office and presidential candidate will have US owned by China!!
@HouseGOP @Sen

Same replacement for Trump

In [20]:
trump_variations = {
    r'\bPresident Donald Trump\b': 'presidential candidate',
    r'\bDonald Trump\b': 'presidential candidate',
    r'\bPresident Trump\b': 'presidential candidate',
    r'\bTrump\b': 'presidential candidate',
    r'\bDJT\b': 'presidential candidate',
}

In [21]:
# compiling regex patterns
patterns = {re.compile(k): v for k, v in trump_variations.items()}

def replace_and_binary(text):
    trump_mentioned = 0
    for pattern, replacement in patterns.items():
        if pattern.search(text):
            trump_mentioned = 1
            text = pattern.sub(replacement, text)
    return trump_mentioned, text

# applying the function to the "text" column
df['Trump initially'], df['text'] = zip(*df['text'].apply(replace_and_binary))


display(df)

,created_at_fuzzy,state_likely_tweeted_from,text,source,display_text_width,lang,favorite_count,retweet_count,is_quote,followers_count,friends_count,listed_count,statuses_count,favourites_count,Biden initially,Trump initially
0,2020-08-13T16:55:49Z,California,LIVE: presidential candidate holds press confe...,Twitter Web App,71,en,0,0,False,3281,4938,4,39349,23299,0,1
1,2020-08-30T00:03:31Z,Tennessee,presidential candidate’s claim the ‘silent maj...,Twitter Web App,205,en,9,2,False,2866,3000,2,23786,30906,0,1
2,2020-10-26T23:16:05Z,New York,#NewHampshire #VoteBlue #VoteBlueToSaveAmerica...,Twitter Web App,267,en,7,4,False,32221,26550,52,304911,146137,1,0
3,2020-08-24T09:54:59Z,Florida,"@davidsirota I'm so confused, when is the medi...",Twitter for Android,262,en,2,0,False,457,375,5,16851,7429,1,0
4,2021-02-04T11:31:07Z,Washington,@Carlos27480217 @DonaldJTrumpJr Yep! He has Ba...,Twitter for Android,88,en,0,0,False,43,177,0,2618,1517,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2021-01-15T18:46:33Z,Georgia,U.S. says Capitol rioters meant to 'capture an...,Twitter for iPhone,126,en,2,0,False,1546,1600,2,5156,3058,0,0
1996,2020-12-21T13:11:53Z,Iowa,@KeithOlbermann Look in Keith’s eyes. preside...,Twitter for iPhone,104,en,0,0,False,28,50,0,758,6974,0,1
1997,2021-02-06T06:35:34Z,North Carolina,@AmyAThatcher I guess the right wingers are pi...,Twitter for Android,125,en,1,0,False,598,512,9,26537,156580,1,0
1998,2020-08-14T07:37:59Z,Iowa,Insane....\n\npresidential candidate calls for...,Twitter for iPad,98,en,1,0,False,31,96,0,1041,2414,1,0


In [23]:
df.to_csv('unbiased_tweet_dataset_200.csv')